<a href="https://colab.research.google.com/github/Addi-11/Depression-chatbot/blob/temporary/preprocessing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import collections
import contractions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import re

Checking out the data

In [10]:
df=pandas.read_csv('data.csv',sep=None, error_bad_lines=False)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  
Skipping line 3800: unexpected end of data


,title,response
0,It kills me to see others in worse positions t...,"I don’t know who you are, but I needed this an..."
1,"I don’t know who you are, but I needed this an...",Its no problem. I just know what its like. I'v...
2,Summer depression,[deleted]
3,[deleted],I think it might be the hot weather for me... ...
4,[Chronically depressed 3+ years] I feel pathet...,It’s nice when people give us compliments. We ...


In [11]:
df['title'][0]

"It kills me to see others in worse positions than I am. You matter. You all matter and it doesn't matter what your depression or suicidal thoughts tell you."

In [12]:
df['response'][0]

'I don’t know who you are, but I needed this and I thank you. I’ve been stuck ina runt for a couple months now. Wanting to just die and let it all be over with. But after reading it, I have a different mindset. Once again thank you kind stranger'

Removing deleted responses and titles

In [13]:
df=df[(df.title!='[deleted]') & (df.response!='[deleted]')]

In [14]:
df.shape

(3750, 2)

In [15]:
df.head()

,title,response
0,It kills me to see others in worse positions t...,"I don’t know who you are, but I needed this an..."
1,"I don’t know who you are, but I needed this an...",Its no problem. I just know what its like. I'v...
4,[Chronically depressed 3+ years] I feel pathet...,It’s nice when people give us compliments. We ...
5,Sick of being sad and lonely,"Dude, I'm terribly sorry for how things have h..."
6,"Dude, I'm terribly sorry for how things have h...",This is probably some of the best advice I hav...


Removing all contractions from the text 

In [18]:
df['title']=df['title'].apply(lambda x:x.replace(x,contractions.fix(x)))

In [19]:
df['response']=df['response'].apply(lambda x:x.replace(x,contractions.fix(x)))

Removing special characters and punctuations

In [20]:
df['title']=df['title'].str.replace("[^a-zA-Z#.,]"," ")

In [21]:
df['response']=df['response'].str.replace("[^a-zA-Z#.,]"," ")

In [22]:
df.head()

,title,response
0,It kills me to see others in worse positions t...,"I do not know who you are, but I needed this a..."
1,"I do not know who you are, but I needed this a...",Its no problem. I just know what its like. I h...
4,Chronically depressed years I feel pathet...,it is nice when people give us compliments. We...
5,Sick of being sad and lonely,"Dude, I am terribly sorry for how things have ..."
6,"Dude, I am terribly sorry for how things have ...",This is probably some of the best advice I hav...


Remove extra space and lower case all the text

In [23]:
df['title']=df['title'].apply(lambda x : x.lower().strip())
df['response']=df['response'].apply(lambda x : x.lower().strip())

In [24]:
df.head()

,title,response
0,it kills me to see others in worse positions t...,"i do not know who you are, but i needed this a..."
1,"i do not know who you are, but i needed this a...",its no problem. i just know what its like. i h...
4,chronically depressed years i feel patheti...,it is nice when people give us compliments. we...
5,sick of being sad and lonely,"dude, i am terribly sorry for how things have ..."
6,"dude, i am terribly sorry for how things have ...",this is probably some of the best advice i hav...


Removing all the spelling mistakes - assuming language english only

In [25]:
# !pip install autocorrect
from autocorrect import Speller

     |████████████████████████████████| 1.8MB 3.4MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.0.0-cp36-none-any.whl size=1811641 sha256=378a6861dba60fdae1a39fb3593ef86db1e4348e688929a3e6f2902465da7d57
  Stored in directory: /root/.cache/pip/wheels/0b/06/bc/e66f28d72bed29591eadc79cebb2e7964ad0282804ab233da3
Successfully built autocorrect


In [26]:
spell=Speller(lang='en')
df['title']=df['title'].apply(lambda x: x.replace(x,spell(x)))

In [27]:
spell=Speller(lang='en')
df['response']=df['response'].apply(lambda x: x.replace(x,spell(x)))

Adding space btw puctuation and text<br>
Adding start and end tags to all the sentences

In [28]:
def preprocess_sentence(w):

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence, so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    
    return w

In [29]:
df['title'] = df['title'].progress_apply(lambda x: preprocess_sentence(x))
df['response'] = df['response'].progress_apply(lambda x: preprocess_sentence(x))
df.head()

100%|██████████| 3750/3750 [00:00<00:00, 15657.87it/s]


,title,response
0,<start> it kills me to see others in worse pos...,"<start> i do not know who you are , but i need..."
1,"<start> i do not know who you are , but i need...",<start> its no problem . i just know what its ...
4,<start> chronically depressed years i feel pat...,<start> it is nice when people give us complim...
5,<start> sick of being sad and lonely <end>,"<start> dude , i am terribly sorry for how thi..."
6,"<start> dude , i am terribly sorry for how thi...",<start> this is probably some of the best advi...


Confirming the changes 

In [31]:
titles = df['title'].values
import string
title=[]
for i in range(len(titles)):
  title.append(titles[i].strip())

In [32]:
responses=df['response'].values
response=[]
for i in range(len(responses)):
    response.append(responses[i].strip())

In [33]:
for i in range(2):
  print('Person 1 : ', title[i])
  print('Person 2 : ', response[i])

Person 1 :  <start> it kills me to see others in worse positions than i am . you matter . you all matter and it does not matter what your depression or suicidal thoughts tell you . <end>
Person 2 :  <start> i do not know who you are , but i needed this and i thank you . i have been stuck in runt for a couple months now . wanting to just die and let it all be over with . but after reading it , i have a different mindset . once again thank you kind stranger <end>
Person 1 :  <start> i do not know who you are , but i needed this and i thank you . i have been stuck in runt for a couple months now . wanting to just die and let it all be over with . but after reading it , i have a different mindset . once again thank you kind stranger <end>
Person 2 :  <start> its no problem . i just know what its like . i have been in this rut for a year , and i hate how many people are in the same , if not a worse position . do not kill yourself . all the things that made you happy in the past , that can s

In [34]:
title_counter = collections.Counter([word for sentence in title for word in sentence.split()])
response_counter = collections.Counter([word for sentence in response for word in sentence.split()])

print('{} Title sentences.'.format(len(title)))
print('{} Title words.'.format(len([word for sentence in title for word in sentence.split()])))
print('{} unique Title words.'.format(len(title_counter)))
print('10 Most common words in the Title:')
print('"' + '" "'.join(list(zip(*title_counter.most_common(10)))[0]) + '"')
print()
print('{} Response sentences.'.format(len(response)))
print('{} Response words.'.format(len([word for sentence in response for word in sentence.split()])))
print('{} unique Response words.'.format(len(response_counter)))
print('10 Most common words in the Response dataset:')
print('"' + '" "'.join(list(zip(*response_counter.most_common(10)))[0]) + '"')

3750 Title sentences.
180862 Title words.
8766 unique Title words.
10 Most common words in the Title:
"." "i" "to" "," "you" "and" "a" "<start>" "<end>" "the"

3750 Response sentences.
291977 Response words.
10643 unique Response words.
10 Most common words in the Response dataset:
"." "i" "," "to" "you" "and" "the" "a" "is" "it"
